<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/lindevanbeers/Lab-2/linde-report-lab-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Linde van Beers**

# **Abstract**

3 methods are implemented in this report: Sparse matrix vector multiplication, QR factorisation, and solving of a linear system. All methods were tested and work. The latter 2 have only been implemented and tested for square matrices. 


#**About the code**

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Author: Linde van Beers, 2020

# Based on a template:
# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this report I show the implementation of 3 functions in linear algebra: Sparse matrix-vector product, QR-factorization and direct solve of a linear system. I used information as presented in the Lecture Notes of the course as guidelines for solving these problems. 



# **Methods**

### Sparse matrix-vector product

This algorithm calculates the matrix vector product $Ax$ where $A$ is written in CRS form. I assumed that the row pointer has an extra element set to the number of non-zero elements in the matrix.

In [0]:
def sparse_MVP(x,A_val,col_idx, row_ptr):
  # Assume that row_ptr has an extra element set to NNZ
  n = len(row_ptr)-1
  b = np.zeros(n)
  for i in range(n):
    for j in range(row_ptr[i], row_ptr[i+1]):
      if col_idx[j] > len(x)-1:
        return "sizes do not match"
      b[i] += A_val[j]*x[col_idx[j]]
  return b

### QR factorization

I implemented the Householder QR factorisation algorithm as described in the lecture notes. 
I assumed A is a square matrix. 

In [0]:
def qr_householder(A):
  n = A.shape[1]
  for k in range(n-1):
    x = A[k:n,k]
    v_k = x.copy()
    v_k[0] = v_k[0]-np.sign(x[0])*np.linalg.norm(x)
    v_k = v_k/np.linalg.norm(v_k)
    for m in range(k,n):
      A[k:n,m] = A[k:n,m]-(2*v_k*(np.dot(v_k.T,A[k:n,m])))  
    Q_k = np.eye(n,n)
    Q_k[k:n,k:n]= Q_k[k:n,k:n]-(2*(np.matmul(v_k,v_k.T))/np.dot(v_k.T,v_k))
    if k==0:
      Q = Q_k
    else:
      Q = np.matmul(Q.T,Q_k.T)
  R=A
  return Q,R

### Direct solve

In order to direct solve $Ax = b$ I first treated $A|b$ as a linear system and performed Gaussian elimination in order to get the system in row echelon form. This yielded an upper triangular matrix $A'$ and corresponding vector $b'$ on which I could then use backward substitution to find $x$, such that $A'x=b'$ and therefore $Ax=b$.
I assumed A is a square matrix.

In [0]:
def direct_solve(A,b):
  n = A.shape[1]
  m = A.shape[0]
  x = np.zeros(n)

  # Put A and b in such a format that A is upper triangular
  if A[0,0]==0:
    for i in range(m):
      if A[i,0] != 0:
        A[0,:] = A[0,:] + A[i,:]/A[i,0]
        b[0] = b[0] + b[i]/A[i,0]
        break
  for i in range(m-1):
    for j in range(i+1,n):
      c = A[j,i]/A[i,i]
      A[j,:] = A[j,:] - c*A[i,:]
      b[j] = b[j] - c*b[i]

  # Use backward substitution to solve
  x[n-1] = b[n-1]/A[n-1,n-1]
  for i in range(n-2,-1,-1):
    sum = 0
    for j in range(i+1, n):
      sum = sum +A[i,j]*x[j]
    x[i] = (b[i]-sum)/A[i,i]
  
  return x

# **Results**

### Sparse matrix-vector product

I tested my function with a correct input and two incorrec inputs. The first, where $b$ is too small for $A$, gives an error as was intended. When $b$ is too large however, the algorithm just assumes that $A$ is also larger but just consists out of 0's in the columns that are 'missing'. This cannot be helped since a sparse matrix does not specify its dimensions. 

In [323]:
A = np.matrix([[0,1,0],[2,0,3],[0,0,4]])
x = [1,2,3]
print("A:",A)
print("x:",x)
# Create a sparse matrix object 
val = np.array([1,2,3,4])
col_idx = np.array([1,0,2,2])
row_ptr = np.array([0,1,3,4])
# Print sparse matrix object attributes
print('Sparse matrix values: \n',val)
print('Sparse matrix column indices: \n',col_idx)
print('Sparse matrix row pointer: \n',row_ptr)
# Calculate b using the sparse matrix
b = sparse_MVP(x, val, col_idx, row_ptr)
# Calculate b using dense matrix 
bAns = A*np.matrix(x).T
print("answer we are looking for:",bAns.T)
print("answer by function:",b)

A = np.matrix([[0,4,0,-4],[2,0,0,3],[0,0,0,5]])
x = [5,7,2]
print("\nA:",A)
print("x:",x)
# Create a sparse matrix object 
val = np.array([4,-4,2,3,5])
col_idx = np.array([1,3,0,3,3])
row_ptr = np.array([0,2,4,5])
# Print sparse matrix object attributes
print('Sparse matrix values: \n',val)
print('Sparse matrix column indices: \n',col_idx)
print('Sparse matrix row pointer: \n',row_ptr)
# Calculate b using the sparse matrix
b = sparse_MVP(x, val, col_idx, row_ptr)
print("answer by function:",b)

A = np.matrix([[0,4,0,-4],[2,0,0,3],[0,0,0,5]])
x = [5,7,2,4,5]
print("\nA:",A)
print("x:",x)
# Create a sparse matrix object 
val = np.array([4,-4,2,3,5])
col_idx = np.array([1,3,0,3,3])
row_ptr = np.array([0,2,4,5])
# Print sparse matrix object attributes
print('Sparse matrix values: \n',val)
print('Sparse matrix column indices: \n',col_idx)
print('Sparse matrix row pointer: \n',row_ptr)
# Calculate b using the sparse matrix
b = sparse_MVP(x, val, col_idx, row_ptr)
print("answer by function:",b)
print('''CSR format has minimum size, not maximum, 
because there could be 0's going each direction. ''')

A: [[0 1 0]
 [2 0 3]
 [0 0 4]]
x: [1, 2, 3]
Sparse matrix values: 
 [1 2 3 4]
Sparse matrix column indices: 
 [1 0 2 2]
Sparse matrix row pointer: 
 [0 1 3 4]
answer we are looking for: [[ 2 11 12]]
answer by function: [ 2. 11. 12.]

A: [[ 0  4  0 -4]
 [ 2  0  0  3]
 [ 0  0  0  5]]
x: [5, 7, 2]
Sparse matrix values: 
 [ 4 -4  2  3  5]
Sparse matrix column indices: 
 [1 3 0 3 3]
Sparse matrix row pointer: 
 [0 2 4 5]
answer by function: sizes do not match

A: [[ 0  4  0 -4]
 [ 2  0  0  3]
 [ 0  0  0  5]]
x: [5, 7, 2, 4, 5]
Sparse matrix values: 
 [ 4 -4  2  3  5]
Sparse matrix column indices: 
 [1 3 0 3 3]
Sparse matrix row pointer: 
 [0 2 4 5]
answer by function: [12. 22. 20.]
CSR format has minimum size, not maximum, 
because there could be 0's going each direction. 


### QR Factorization

My algorithm works on square matrices. The second test does give some small errors because the program rounds 1/3 and 2/3, introducing some computational error.

In [328]:
A = np.matrix([[12,-51,4],[6,167,-68],[-4,24,-41]])
print("A",A)
Q,R = qr_householder(A)
print("Q",Q)
print("R",R)
print("QR", np.matmul(Q,R))

print("\n")
A = np.matrix([[2.,-2.,18],[2.,1.,0.],[1.,2.,0.]])
print("A",A)
Q,R = qr_householder(A)
print("Q",Q)
print("R",R)
print("QR", np.matmul(Q,R))

A [[ 12 -51   4]
 [  6 167 -68]
 [ -4  24 -41]]
Q [[ 0.85714286  0.39428571 -0.33142857]
 [ 0.42857143 -0.90285714  0.03428571]
 [-0.28571429 -0.17142857 -0.94285714]]
R [[  14   21  -14]
 [   0 -175   70]
 [   0    0   35]]
QR [[ 12. -51.   4.]
 [  6. 167. -68.]
 [ -4.  24. -41.]]


A [[ 2. -2. 18.]
 [ 2.  1.  0.]
 [ 1.  2.  0.]]
Q [[ 0.66666667  0.66666667 -0.33333333]
 [ 0.66666667 -0.33333333  0.66666667]
 [ 0.33333333 -0.66666667 -0.66666667]]
R [[ 3.00000000e+00  4.44089210e-16  1.20000000e+01]
 [-4.44089210e-16 -3.00000000e+00  1.20000000e+01]
 [-2.22044605e-16  4.44089210e-16 -6.00000000e+00]]
QR [[ 2.00000000e+00 -2.00000000e+00  1.80000000e+01]
 [ 2.00000000e+00  1.00000000e+00 -4.73695157e-15]
 [ 1.00000000e+00  2.00000000e+00 -2.59052039e-15]]


### Direct Solve Ax = b

The algorithm works on square matrices as shown by the following test:

In [335]:
A = np.matrix([[1.,1.,-1.],[1.,-1.,2.],[2.,1.,1.]])
b = np.array([7.,3.,9.])
print("A:",A)
print("b:",b)
x = direct_solve(A,b)
A = np.matrix([[1.,1.,-1.],[1.,-1.,2.],[2.,1.,1.]])
print("x:",x)
print("Ax:", np.matmul(A,np.matrix(x).T))

print("\n")
A = np.matrix([[0.,4.,1.],[1.,3.,-2.],[-1.,-2.,2.]])
b = np.array([2.,1.,3.])
print("A:",A)
print("b:",b)
x = direct_solve(A,b)
A = np.matrix([[0.,4.,1.],[1.,3.,-2.],[-1.,-2.,2.]])
print("x:",x)
print("Ax:", np.matmul(A,np.matrix(x).T))


A: [[ 1.  1. -1.]
 [ 1. -1.  2.]
 [ 2.  1.  1.]]
b: [7. 3. 9.]
x: [ 6. -1. -2.]
Ax: [[7.]
 [3.]
 [9.]]


A: [[ 0.  4.  1.]
 [ 1.  3. -2.]
 [-1. -2.  2.]]
b: [2. 1. 3.]
x: [-39.   4. -14.]
Ax: [[2.]
 [1.]
 [3.]]


# **Discussion**

All methods have been implemented to a point that they work well. Unfortunately, due to illness, I did not have the time or energy to take it further and make sure the methods also work on matrices that are not square, which would have been a good addition, since I do have ideas on how to make this work. 